<a href="https://colab.research.google.com/github/bolinwong/Mobilenet_maskrcnn/blob/main/Mobilenet_maskrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/bolinwong/Mobilenet_maskrcnn.git
%cd Mobilenet_maskrcnn
!pip3 install -r requirements.txt
!python3 coco.py install

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip uninstall keras-nightly

In [ ]:
!pip install h5py==2.10.0
!pip install q keras==2.1.6

In [ ]:
import keras
print(keras.__version__)

In [ ]:
%tensorflow_version 1.x

In [ ]:
# python 2 compability
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Import Python Packages
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import imgaug
import json
import pandas as pd
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Import Mobile Mask R-CNN Packages
from mmrcnn.config import Config
from mmrcnn import utils
import mmrcnn.model as modellib
from mmrcnn import visualize
from mmrcnn.model import log


%matplotlib inline 


In [ ]:
 #Root directory of the project
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/mobilenet_contamination"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mobile_mask_rcnn_coco.h5")
#COCO_MODEL_PATH = os.path.join(r"D:\Dokumen Pengguna\Desktop\Label_Me\mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
 #   utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    ## Give the configuration a recognizable name
    NAME = "contamination"

    ## GPU
    IMAGES_PER_GPU = 1
    GPU_COUNT = 2

    ## Number of classes (including background)
    NUM_CLASSES = 1 + 1  # COCO has 80 classes (1+80)

    ## Backbone Architecture
    BACKBONE = "mobilenetv1"
    
    BATCH_SIZE = 2
    STEPS_PER_EPOCH = 219
    ## Size Options
    BACKBONE_STRIDES = [4, 8, 16, 32, 64] #ResNet
    #BACKBONE_STRIDES = [2, 4, 8, 16, 32]

    #RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256) #ResNet
    RPN_ANCHOR_SCALES = (8 , 16, 32, 64, 128)

    MINI_MASK_SHAPE = (56, 56) #ResNet
    #MINI_MASK_SHAPE = (28, 28)

 

    #TRAIN_ROIS_PER_IMAGE = 200 #ResNet
    #TRAIN_ROIS_PER_IMAGE = 128

    
config = CocoConfig()
config.display()

In [ ]:
class MicrocontrollerDataset(utils.Dataset):
    def load_dataset(self, dataset_dir):
        self.add_class('dataset', 1, 'contamination')
        
        
        #assert subset in ["train", "val"]
       # dataset_dir = os.path.join(dataset_dir, subset)
        
        # find all images
        for i, filename in enumerate(os.listdir(dataset_dir)):
            if '.png' in filename:
                self.add_image('dataset', 
                               image_id=i, 
                               path=os.path.join(dataset_dir, filename), 
                               annotation=os.path.join(dataset_dir, filename.replace('.png', '.json')))
            
    def extract_masks(self, filename):
        json_file = os.path.join(filename)
        with open(json_file) as f:
            img_anns = json.load(f)
            
        masks = np.zeros([700, 1000, len(img_anns['shapes'])], dtype='uint8')
        classes = []
        for i, anno in enumerate(img_anns['shapes']):
            mask = np.zeros([700, 1000], dtype=np.uint8)
            cv2.fillPoly(mask, np.array([anno['points']], dtype=np.int32), 1)
            masks[:, :, i] = mask
            classes.append(self.class_names.index(anno['label']))
        return masks, classes
 
    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load XML
        masks, classes = self.extract_masks(path)
        return masks, np.asarray(classes, dtype='int32')
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
# Create training and validation set
# train set
dataset_train = MicrocontrollerDataset()
dataset_train.load_dataset(r"/content/drive/MyDrive/Contamination2/Train")
dataset_train.prepare()
print('Train: %d' % len(dataset_train.image_ids))
 
# test/val set
dataset_val = MicrocontrollerDataset()
dataset_val.load_dataset(r"/content/drive/MyDrive/Contamination2/Test")
dataset_val.prepare()
print('Validation: %d' % len(dataset_val.image_ids))

dataset_test = MicrocontrollerDataset()
dataset_test.load_dataset(r"/content/drive/MyDrive/Contamination2/Test")
dataset_test.prepare()
print('Test: %d' % len(dataset_test.image_ids))


In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 3)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

Augmentation


In [ ]:


# Image augmentation (light but constant)
augmentation = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-45, 45),
            
        ),

    ]),
    
    iaa.Flipud(0.2), # vertically flip 20% of all images
    iaa.Crop(percent=(0, 0.2)),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen

        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

# test on the same image as above
imggrid = augmentation.draw_grid(image[:, :, 0], cols=5, rows=2)
plt.figure(figsize=(30, 12))
_ = plt.imshow(imggrid[:, :, 0],cmap='gray')



In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_weights = "imagenet"  # imagenet, coco, or last

if init_weights == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_weights == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(DEFAULT_WEIGHTS, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_weights == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [ ]:
LEARNING_RATE = 0.006

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")


In [ ]:
%%time
## train heads with higher lr to speedup the learning
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE*2,
            epochs=2,
            layers='heads',
            augmentation=None)  ## no need to augment yet

history = model.keras_model.history.history

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE,
            epochs=6,
            layers='all',
            augmentation=augmentation)

new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE/5,
            epochs=20,
            layers='all',
            augmentation=augmentation)

new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
epochs = range(1,len(next(iter(history.values())))+1)
pd.DataFrame(history, index=epochs)

In [ ]:

plt.figure(figsize=(17,5))

plt.subplot(131)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train class ce")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid class ce")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid box loss")
plt.legend()

plt.show()


In [ ]:

best_epoch = np.argmin(history["val_loss"])
print("Best Epoch:", best_epoch + 1, history["val_loss"][best_epoch])


In [ ]:
model.keras_model.summary()

INFERENCE

In [ ]:
class InferenceConfig(CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")

model_path = r"/content/drive/MyDrive/Colab Notebooks/mobilenet_contamination/logs/1024_contamination20210817T1030/mask_rcnn_1024_contamination_0020.h5"
sys.path.append(model_path)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


In [ ]:
# Load trained weights
final_model_path = model_path

print("Loading weights from ", final_model_path)
model.load_weights(final_model_path, by_name=True)

In [ ]:
class DieSolFrameDatasetTwo(utils.Dataset):
    def load_dataset(self, dataset_dir):
        self.add_class('dataset', 1, 'contamination')

        #assert subset in ["train", "val"]
       # dataset_dir = os.path.join(dataset_dir, subset)
        
        # find all images
        for i, filename in enumerate(os.listdir(dataset_dir)):
            if '.png' in filename:
                self.add_image('dataset', 
                               image_id=i, 
                               path=os.path.join(dataset_dir, filename), 
                               annotation=os.path.join(dataset_dir, filename.replace('.png', '.json')))

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

Ground Truth Label

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
import pandas as pd
import numpy as np
import os 

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = dataset_test.image_ids
APs = []
precisions_arr = []
recalls_arr = []
overlaps_arr = []
class_ids_arr = []
scores_arr = []

#ground-truth and predictions lists
gt_tot = np.array([])
pred_tot = np.array([])

for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
        
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)

    #check if the vectors len are equal
    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    
    APs.append(AP)
    
    #Append Precision
    for precision in precisions:
      precisions_arr.append(precision)

    #Append recalls
    for recall in recalls:
      recalls_arr.append(recall)

    #Append overlaps
    for overlap in overlaps:
      overlaps_arr.append(overlap)

    #Append clas_ids
    for class_id in r["class_ids"]:
      class_ids_arr.append(class_id)

    #Append scores
    for score in r["scores"]:
      scores_arr.append(score)

    print("The actual mean average precision for the whole images (matterport methode): ", sum(APs)/len(APs))
 
gt_tot=gt_tot.astype(int)
pred_tot=pred_tot.astype(int)
#save the vectors of gt and pred
save_dir = "output"
gt_pred_tot_json = {"gt_tot" : gt_tot, "pred_tot" : pred_tot}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_test.json"))
    
#print the confusion matrix and compute true postives, false positives and false negative for each class: 
#ps : you can controle the figure size and text format by choosing the right values
tp, fp, fn = utils.plot_confusion_matrix_from_data(gt_tot, pred_tot, dataset_test.class_names, fz=18, figsize=(20,20), lw=0.5)

print("mAP: ", np.mean(APs))



In [ ]:


print("tp for each class :",tp)
print("fp for each class :",fp)
print("fn for each class :",fn)

#eliminate the background class (class A) from tps fns and fns lists since it doesn't concern us anymore : 
del tp[0]
del fp[0]
del fn[0]
print("\n########################\n")
print("tp for each class :",tp)
print("fp for each class :",fp)
print("fn for each class :",fn)

print("\n########################\n")
#Precision 
Precision = np.array(tp)/(np.array(tp)+ np.array(fp))
print("Precision: " ,np.mean(Precision))

#Recall 
Recall = np.array(tp)/(np.array(tp)+ np.array(fn))
print("Recall: ",np.mean(Recall))

#F1 Score
f1 = (2*Precision*Recall)/(Precision + Recall)
print("F1 Score: ", np.mean(f1))

In [ ]:
visualize.plot_precision_recall (AP, precisions, recalls)

In [ ]:
loop_count = 5
results = model.detect([original_image], verbose=1)

start_time = time.time()
for i in range(loop_count):    
    r = results[0]
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_test.class_names, r['scores'], ax=get_ax())
print("Keras inferences with %s second in average" %((time.time() - start_time) / loop_count))
#r = results[0]
#visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
 #                           dataset_test.class_names, r['scores'], ax=get_ax())